In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aiml-general-championship:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71608%2F7895811%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240731%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240731T160003Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D96c5df5c9599d65577fdc4bd9081f0110dc50c675b0872c7913c5c9f05300cb0ebc74c79f07112231e488864d1c9ba1c0e7d9e72732033640ec443ecf3d0663434587c6590c586e34e4bf578a6b73c46b8f35ab28bf7f9a7792bf9e7e925ddf5ea1294e979536c888f6c066979cbb9fab480ae49a52d9b2b757a6c4eb01a2db472c047cb127c0e2db2ccd400d9ac441b93194411e3c0535ee581a40288b6d9e4a8bbe5036fdfd7691d039f0873a125c48c887b35ca418352079a8499ab6eb54aee3dda514ce4373208d9c14fa4865cff045c92bbc182e85c5944030cea63a84dd1dc8b8bf44057e14d6f3a70d514833b3003686a1769b709bbed20e50cfc1739'

KAGGLE_WORKING_PATH = '/kaggle/working'
CHUNK_SIZE = 40960

# Ensure directories exist in writable paths
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path.split('/')[-1]
    destination_path = os.path.join(KAGGLE_WORKING_PATH, directory)

    try:
        with urlopen(download_url) as response, NamedTemporaryFile(delete=False) as tfile:
            total_length = response.headers.get('content-length')
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            while True:
                data = response.read(CHUNK_SIZE)
                if not data:
                    break
                tfile.write(data)
                dl += len(data)
                done = int(50 * dl / int(total_length)) if total_length else 0
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50 - done)}] {dl} bytes downloaded")
                sys.stdout.flush()
            print()

            # Close the temporary file so it can be accessed by the extraction methods
            tfile.close()

            # Extract the downloaded file
            if filename.endswith('.zip'):
                with ZipFile(tfile.name) as zfile:
                    zfile.extractall(destination_path)
            else:
                with tarfile.open(tfile.name) as tarfile_obj:
                    tarfile_obj.extractall(destination_path)

            print(f'Downloaded and uncompressed: {directory}')
    except HTTPError:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
    except OSError:
        print(f'Failed to load {download_url} to path {destination_path}')

print('Data source import complete.')


In [ ]:
%matplotlib inline

# Python libraries
import os
import cv2
import itertools
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

# PyTorch libraries
import torch
from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

# Scikit-learn libraries
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Ensure results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)


In [ ]:
data_dir = "/kaggle/input/aiml-general-championship"
print(os.listdir(data_dir))

<h1>Loading Data... </h1>

In [ ]:
# Define the directory containing the images
img_data_dir = os.path.join(data_dir, "KCDH2024_Training_Input_10K", "KCDH2024_Training_Input_10K")

# Get a list of all image file paths with a .jpg extension
all_image_path = glob(os.path.join(img_data_dir, '*.jpg'))

# Create a dictionary mapping image filenames to their paths
imageid_path_dict = {
    os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path
}

# Define a dictionary mapping lesion types to their descriptions
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Dermatofibroma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}


In [ ]:
len(imageid_path_dict)

In [ ]:
img = cv2.imread(imageid_path_dict["ISIC_0024308"])
img.shape

In [ ]:
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 224, 224              # Size to resize..
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

In [ ]:
# means , stdevs = compute_img_mean_std(all_image_path)

In [ ]:
# Store Values to save time in future..
norm_mean = [0.76696384, 0.54525656, 0.56884694]
norm_std = [0.13945772, 0.15192385, 0.16916788]

In [ ]:
lesion_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_LesionGroupings.csv'))
truth_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_GroundTruth.csv'))

df = pd.merge(lesion_db, truth_db, on = 'image', how = 'inner')
df.drop("diagnosis_confirm_type", axis = 1, inplace = True)
df['path'] = df['image'].map(imageid_path_dict.get)
df

In [ ]:
# Remove the rows not containg path to images..
df = df[df['path'].notna()]
df

In [ ]:
# Convert 7 different columns of different lesisons to single ...

cell_type_idx = []

for index, row in df.iterrows():
    cell_type_idx_row = row["MEL"], row["NV"], row["BCC"], row["AKIEC"], row["BKL"], row["DF"], row["VASC"]
    cell_type_idx.append(cell_type_idx_row.index(1))

# Assign a new column..
df["cell_type_idx"] = cell_type_idx

# Drop older columns..
df.drop( columns = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace = True)

df.head()

In [ ]:
# Determine how many images are associated with each lesion_id ?
df_undup = df.groupby('lesion_id').count()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup.head()

In [ ]:
df_undup = df.groupby('lesion_id').count()
df_undup.head()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup[:]

In [ ]:
# Identify lesion_id's that have duplicate images and those that have only one image.
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# Create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['lesion_id']
# Apply the function to this new column
df['duplicates'] = df['duplicates'].apply(get_duplicates)
df.head()

In [ ]:
df['duplicates'].value_counts()

In [ ]:
# Filter out images that don't have duplicates (count = 1)  (Removed augmented images)
# We will use this data to get validation set..
df_undup = df[df['duplicates'] == 'unduplicated']
df_undup.shape

In [ ]:
# Create a val set using df as none of these images have augmented duplicates in the training set now..
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

In [ ]:
df_val['cell_type_idx'].value_counts()

In [ ]:
# Remove the validation rows from the original data to get training rows..

# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# Identify train and val rows..
# Create a new colum that is a copy of the image column
df['train_or_val'] = df['image']

# Apply the function to this new column
df['train_or_val'] = df['train_or_val'].apply(get_val_rows)

# Filter out training rows
df_train = df[df['train_or_val'] == 'train']
df_train = df_train.drop('train_or_val', axis = 1, inplace = False)
print(len(df_train))
print(len(df_val))

In [ ]:
df_train['cell_type_idx'].value_counts()

# Duplicate rows to balance the number of rows in 7 classes..
data_aug_rate = [6.3,1.025,13,19.8,6.1,56.4,47.7]

# Iterate over unique values of 'cell_type_idx'
for i in df_train['cell_type_idx'].unique():

    if data_aug_rate[i] > 1:

        # Filter the DataFrame for the current value of 'cell_type_idx'
        filtered_df = df_train[df_train['cell_type_idx'] == i]

        # Duplicate rows based on the data augmentation rate for this value of 'cell_type_idx'
        duplicated_rows = filtered_df.sample(frac=data_aug_rate[i] - 1, replace=True)

        # Concatenate the original DataFrame with the duplicated rows
        df_train = pd.concat([df_train, duplicated_rows], ignore_index=True)

df_train['cell_type_idx'].value_counts()

<h1>DataSet Spliiting</h1>

In [ ]:
len(df_train)

In [ ]:
# Split the test set again in a validation set and a true test set:

df_val, df_test = train_test_split(df_val, test_size=0.5)
df_train = df_train.reset_index()
df_val = df_val.reset_index()
df_test = df_test.reset_index()

In [ ]:
print(len(df_test))
df_test['cell_type_idx'].value_counts()

<h1>Define Model</h1>

In [ ]:
# feature_extract is a boolean that defines finetuning or feature extracting.
# If feature_extract = False, the model is finetuned and all model parameters are updated.
# If feature_extract = True, only the last layer parameters are updated, the others remain fixed.

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set
    model_ft = None
    input_size = 0

    if model_name == "efficientnet":
        model_ft = models.efficientnet_b3(pretrained=True, progress=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
torch.cuda.is_available()

In [ ]:
# Define a new model variable..

model_name = "efficientnet"
num_classes = 7
feature_extract = False
# Initialize the model
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Define the device:
device = torch.device('cuda:0')
# device = torch.device('cpu') # If using cpu
# Put the model on the device:
model = model_ft.to(device)

<H1>Augmenting Data...</H1>

In [ ]:
# Define the transformation for training images
train_transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# Define the transformation for validation images
val_transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# Define the transformation for test images
test_transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])


In [ ]:
# Define a pytorch dataloader for dataset..
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [ ]:
# Define the training set using the table train_df and using the defined transitions (train_transform)
training_set = HAM10000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)

# Same for the validation set:
validation_set = HAM10000(df_val, transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=32, shuffle=False, num_workers=4)

# Same for the test set:
test_set = HAM10000(df_test, transform=train_transform)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# this function is used during training process, to calculate the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# Initialize lists to store total training loss and accuracy
total_loss_train = []
total_acc_train = []

def train(train_loader, model, criterion, optimizer, epoch, threshold_batch=None):
    """
    Train the model for one epoch.

    Args:
        train_loader (DataLoader): DataLoader for training data.
        model (nn.Module): The model to be trained.
        criterion (nn.Module): Loss function.
        optimizer (Optimizer): Optimizer for updating model parameters.
        epoch (int): Current epoch number.
        threshold_batch (int, optional): Maximum number of batches to process.

    Returns:
        float: Average training loss.
        float: Average training accuracy.
    """
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)

    for i, data in enumerate(train_loader):
        # Break if the batch index exceeds the threshold
        if threshold_batch and i >= threshold_batch:
            break

        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # Compute loss and update model
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute accuracy
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item() / N)
        train_loss.update(loss.item())

        curr_iter += 1

        # Print progress every 100 iterations
        if (i + 1) % 100 == 0:
            print(f'[Epoch {epoch}], [Iter {i + 1} / {len(train_loader)}], '
                  f'[Train Loss {train_loss.avg:.5f}], [Train Acc {train_acc.avg:.5f}]')
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)

    return train_loss.avg, train_acc.avg


In [ ]:
def validate(val_loader, model, criterion, epoch):
    """
    Validate the model on the validation dataset.

    Args:
        val_loader (DataLoader): DataLoader for validation data.
        model (nn.Module): The model to be evaluated.
        criterion (nn.Module): Loss function.
        epoch (int): Current epoch number.

    Returns:
        list: List of validation loss values.
        list: List of validation accuracy values.
    """
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()

    total_val_loss = []
    total_val_acc = []

    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader), desc="Validating"):

            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            # Update validation accuracy and loss
            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item() / N)
            val_loss.update(criterion(outputs, labels).item())

            # Store metrics every 3 iterations
            if (i + 1) % 3 == 0:
                total_val_acc.append(val_acc.avg)
                total_val_loss.append(val_loss.avg)

    print('------------------------------------------------------------')
    print(f'[Epoch {epoch}], [Val Loss {val_loss.avg:.5f}], [Val Acc {val_acc.avg:.5f}]')
    print('------------------------------------------------------------')

    return total_val_loss, total_val_acc


In [ ]:
# Start Training..
def train_model(epoch_num, lr, optimizer_func = optim.Adam, threshold_batch = None, criterion = nn.CrossEntropyLoss()):

    best_val_acc = 0
    total_loss_val, total_acc_val = [],[]
    for epoch in tqdm(range(1, epoch_num+1)):

        # set optimizer and loss function
        optimizer = optimizer_func(model.parameters(), lr= lr)
        criterion = criterion.to(device)

        loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch, threshold_batch)
        loss_val, acc_val = validate(val_loader, model, criterion , optimizer, epoch)
        total_loss_val += loss_val
        total_acc_val += acc_val
    return total_loss_val, total_acc_val

In [ ]:
# Store Training data..
total_loss_val, total_acc_val = [],[]
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam

In [ ]:
# Training - 1st
data = train_model(1,1e-3)
total_loss_val += data[0]
total_acc_val += data[1]

In [ ]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

In [ ]:
# Training - 2nd..
train_model(1,1e-5,threshold_batch = 300)

In [ ]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

In [ ]:
# Training - 3rd..
train_model(1,1e-5,threshold_batch = 300)

In [ ]:
# Check results on test dataset..
validate(test_loader, model, criterion, optimizer, 1)

<h1> Analaysis Of Accuracies.. </h1>

In [ ]:
total_loss_val

In [ ]:
# Analysing accuracy and loss of validation data over the model..

fig = plt.figure(num = 1)
fig2 = fig.add_subplot(1,1,1)
fig2.plot(total_acc_val, label = 'validation accuracy')
fig2.plot(total_loss_val, label = 'validation loss')

plt.legend()
plt.show()

In [ ]:
# Analysing accuracy and loss of training data over the model..

fig = plt.figure(num=1)
fig1 = fig.add_subplot(1,1,1)
fig1.plot(total_acc_train, label = 'training accuracy')
fig1.plot(total_loss_train, label = 'training loss')

plt.legend()
plt.show()


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Validation data evaluation

# Set model to evaluation mode
model.eval()

# Initialize lists to store true labels and predictions
y_label = []
y_predict = []

# Disable gradient computation
with torch.no_grad():
    for i, data in enumerate(val_loader):
        images, labels = data
        images = Variable(images).to(device)
        
        # Forward pass
        outputs = model(images)
        prediction = outputs.max(1, keepdim=True)[1]

        # Append labels and predictions to the lists
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(prediction.cpu().numpy().squeeze())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(y_label, y_predict)

# Define labels for plotting
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc', 'mel']

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, plot_labels)


In [ ]:
# Test data evaluation

# Set model to evaluation mode
model.eval()

# Initialize lists to store true labels and predictions
test_y_label = []
test_y_predict = []

# Disable gradient computation
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = Variable(images).to(device)
        
        # Forward pass
        outputs = model(images)
        prediction = outputs.max(1, keepdim=True)[1]
        
        # Append labels and predictions to the lists
        test_y_label.extend(labels.cpu().numpy())
        test_y_predict.extend(prediction.cpu().numpy().squeeze())

# Compute the confusion matrix for the test data
confusion_mtx_test = confusion_matrix(test_y_label, test_y_predict)

# Define labels for plotting
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc', 'mel']

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx_test, plot_labels)


In [ ]:
# Generate a validation classification report
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)

In [ ]:
# Generate a test classification report
report = classification_report(test_y_label, test_y_predict, target_names=plot_labels)
print(report)

In [ ]:
# True vs Incoorect classified analysis for validation data

label_frac_error = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

In [ ]:
# True vs Incoorect classified analysis for test data

label_frac_error = 1 - np.diag(confusion_mtx_test) / np.sum(confusion_mtx_test, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

<h1> Predictions... </h1>

In [ ]:
# Get data..
img_data_dir = data_dir + r"/KCDH2024_Test_Input/KCDH2024_Test_Input"
all_image_path = glob(os.path.join(img_data_dir, '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}  # key - image filename,   value - path to image
len(imageid_path_dict)

In [ ]:
# Header of txt file..
text = "ID,Class\n"

# Rows ..
model.eval()
with torch.no_grad():
    for imagename, path in tqdm(imageid_path_dict.items()):
         X = Image.open(path)
         X = test_transform(X)
         X = Variable(X).to(device)
         output = model(X.unsqueeze(0))
         prediction = output.max(1, keepdim=True)[1].item()
         text += f"{imagename},{prediction}\n"

In [ ]:
with open(f'predictions.csv', 'w') as txtfile:
    txtfile.write(text)